# Mercedes-Benz Greener Manufacturing

#### Problem Statement : Reduce the time a Mercedes-Benz spends on the test bench.

Following actions should be performed:

* If for any column(s), the variance is equal to zero, then you need to remove those variable(s).


* Check for null and unique values for test and train sets.


* Apply label encoder.


* Perform dimensionality reduction.


* Predict your test_df values using XGBoost.

In [1]:
!git clone https://github.com/Simplilearn-Edu/Machine-Learning--Projects.git

fatal: destination path 'Machine-Learning--Projects' already exists and is not an empty directory.


In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')
train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [3]:
test = pd.read_csv('test.csv')
test.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0


### If for any column(s), the variance is equal to zero, then you need to remove those variable(s).

## Find cardinality of column

1. Create an empty list of 3 lists

lst = [[1],[2],[other]]

2. iterate through all the columns

3. In iterator:
    
    a. find type of all the columns
    
    b. find # of unique values 
    
        i. if unique value = 1 then the variance is constant - assign to 1
        ii. if unique value = 2 then binary column - assign to 2\
        iii. else other

4. Drop 1

In [4]:
# Count the data in each of the columns

cols = [c for c in train.columns if 'X' in c]

counts = [[], [], []]
for c in cols:
    typ = train[c].dtype
    uniq = len(np.unique(train[c]))
    if uniq == 1:
        counts[0].append(c)
    elif uniq == 2 and typ == np.int64:
        counts[1].append(c)
    else:
        counts[2].append(c)

print(' Constant features: {}\n Binary features: {}\n Categorical features: {}\n'
      .format(*[len(c) for c in counts]))
print('Constant features:', counts[0])
#print('Binary features:', counts[1])
print('Categorical features:', counts[2])

 Constant features: 12
 Binary features: 356
 Categorical features: 8

Constant features: ['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347']
Categorical features: ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']


In [5]:
print(train.shape)
print(test.shape)

(4209, 378)
(4209, 377)


In [6]:
usable_columns = list(set(train.columns) - set(['ID', 'y']))
for column in usable_columns:
    cardinality = len(np.unique(train[column]))
    if cardinality == 1:
        train.drop(column, inplace = True, axis=1) #Column with only one,value is useless so we drop it
        test.drop(column, inplace = True, axis=1)

In [7]:
print(train.shape)
print(test.shape)

(4209, 366)
(4209, 365)


### Apply label encoder.

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

In [10]:
print(f' Shape of train : {train.shape} \n Shape of test : {test.shape}')

 Shape of train : (4209, 366) 
 Shape of test : (4209, 365)


### Perform dimensionality reduction.

In [11]:
from sklearn.decomposition import PCA, FastICA

In [12]:
## PCA

pca = PCA(n_components=10,random_state=42)
pca_results_train = pca.fit_transform(train.drop(['y'],axis=1))
pca_results_test = pca.fit_transform(test)

In [13]:
# ICA

ica = FastICA(n_components=10,random_state=42)
ica_results_train = ica.fit_transform(train.drop(['y'],axis=1))
ica_results_test = ica.fit_transform(test)

In [14]:
# Append decomposition to datasets

for i in range(1,11):
    train['pca_' + str(i)] = pca_results_train[:,i-1]
    test['pca_' + str(i)] = pca_results_test[:,i-1]

    train['ica_' + str(i)] = ica_results_train[:,i-1]
    test['ica_' + str(i)] = ica_results_test[:,i-1]

y_train = train['y']
y_mean = np.mean(y_train)

### Working Predict your test_df values using XGBoost.

In [15]:
import xgboost as xgb

In [16]:
xgb_parama = {
    'n_trees':500,
    'eta':0.005,
    'max_depth':4,
    'subsample':0.95,
    'objective':'reg:linear',
    'eval_metric':'rmse',
    'base_score':y_mean,
    'silent':1
}

In [17]:
## Creating training and test Dmatrix

dtrain = xgb.DMatrix(train.drop(['y'],axis=1),y_train)
dtest = xgb.DMatrix(test)

In [18]:
cv_result = xgb.cv(xgb_parama,
                  dtrain,
                  num_boost_round=1000, 
                   early_stopping_rounds=50,
                   verbose_eval=50,
                   show_stdv=False
                  )

[0]	train-rmse:12.63990	test-rmse:12.63861
[50]	train-rmse:11.08371	test-rmse:11.16321
[100]	train-rmse:10.00694	test-rmse:10.17075
[150]	train-rmse:9.27608	test-rmse:9.51671
[200]	train-rmse:8.78730	test-rmse:9.10002
[250]	train-rmse:8.46048	test-rmse:8.83959
[300]	train-rmse:8.23462	test-rmse:8.67920
[350]	train-rmse:8.05648	test-rmse:8.58412
[400]	train-rmse:7.90675	test-rmse:8.53977
[450]	train-rmse:7.77614	test-rmse:8.52505
[500]	train-rmse:7.66296	test-rmse:8.52644


In [19]:
num_boost_rounds = len(cv_result)

In [20]:
## train model

model = xgb.train(dict(xgb_parama,silent=0),dtrain,num_boost_round=num_boost_rounds)

[20:43:27] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[20:43:27] WARNING: /workspace/src/learner.cc:328: 
Parameters: { n_trees, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [21]:
from sklearn.metrics import r2_score

print(f'Training r2 : {r2_score(dtrain.get_label(),model.predict(dtrain))}')

Training r2 : 0.6063701393715053


## Predict your test_df values using XGBoost.

In [22]:
y_pred = model.predict(dtest)

In [23]:
test_df_pred = pd.DataFrame({'id':test['ID'].astype(np.int32),'y':y_pred})

In [24]:
test_df_pred.head()

,id,y
0,1,81.783340
1,2,97.855118
2,3,80.438736
3,4,81.771431
4,5,111.733170
